System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores
Environment:
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2023spring/slides/07-algo`


Status `~/Documents/github.com/ucla-biostat-257/2023spring/slides/07-algo/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


## Definition

* Algorithm is loosely defined as a set of instructions for doing something. Input $\to$ Output.

* [Knuth](https://en.wikipedia.org/wiki/The_Art_of_Computer_Programming): (1) finiteness, (2) definiteness, (3) input, (4) output, (5) effectiveness.

## Measure of efficiency

* A basic unit for measuring algorithmic efficiency is **flop**.  

> A flop (**floating point operation**) consists of a floating point addition, subtraction, multiplication, division, or comparison, and the usually accompanying fetch and store.  

    Some books count multiplication followed by an addition (fused multiply-add, FMA) as one flop. This results a factor of up to 2 difference in flop counts.

* How to measure efficiency of an algorithm? Big O notation. If $n$ is the size of a problem, an algorithm has order $O(f(n))$, where the leading term in the number of flops is $c \cdot f(n)$. For example,
    - matrix-vector multiplication `A * b`, where `A` is $m \times n$ and `b` is $n \times 1$, takes $2mn$ or $O(mn)$ flops  
    - matrix-matrix multiplication `A * B`, where `A` is $m \times n$ and `B` is $n \times p$, takes $2mnp$ or $O(mnp)$ flops

* A hierarchy of computational complexity:  
    Let $n$ be the problem size.
    - Exponential order: $O(b^n)$ (NP-hard="horrible")    
    - Polynomial order: $O(n^q)$ (doable)  
    - $O(n \log n )$ (fast)  
    - Linear order $O(n)$ (fast)  
    - Log order $O(\log n)$ (super fast)  
    
* Classification of data sets by [Huber](http://link.springer.com/chapter/10.1007%2F978-3-642-52463-9_1).

| Data Size | Bytes     | Storage Mode          |
|-----------|-----------|-----------------------|
| Tiny      | $10^2$    | Piece of paper        |
| Small     | $10^4$    | A few pieces of paper |
| Medium    | $10^6$ (megatbytes)    | A floppy disk         |
| Large     | $10^8$    | Hard disk             |
| Huge      | $10^9$ (gigabytes)   | Hard disk(s)          |
| Massive   | $10^{12}$ (terabytes) | RAID storage          |

* Difference of $O(n^2)$ and $O(n\log n)$ on massive data. Suppose we have a teraflop supercomputer capable of doing $10^{12}$ flops per second. For a problem of size $n=10^{12}$, $O(n \log n)$ algorithm takes about 
$$10^{12} \log (10^{12}) / 10^{12} \approx 27 \text{ seconds}.$$ 
$O(n^2)$ algorithm takes about $10^{12}$ seconds, which is approximately 31710 years!

* QuickSort and FFT (invented by statistician John Tukey!) are celebrated algorithms that turn $O(n^2)$ operations into $O(n \log n)$. Another example is the Strassen's method, which turns $O(n^3)$ matrix multiplication into $O(n^{\log_2 7})$.

* One goal of this course is to get familiar with the flop counts for some common numerical tasks in statistics.   

> **The form of a mathematical expression and the way the expression should be evaluated in actual practice may be quite different.**

* For example, compare flops of the two mathematically equivalent expressions: `(A * B) * x` and `A * (B * x)` where `A` and `B` are matrices and `x` is a vector.

In [3]:
using BenchmarkTools, Random

Random.seed!(123) # seed
n = 1000
A = randn(n, n)
B = randn(n, n)
x = randn(n)

# complexity is n^3 + n^2 = O(n^3)
@benchmark ($A * $B) * $x

BenchmarkTools.Trial: 658 samples with 1 evaluation.
 Range (min … max):  6.379 ms … 20.125 ms  ┊ GC (min … max): 0.00% … 5.68%
 Time  (median):     7.089 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.600 ms ±  1.213 ms  ┊ GC (mean ± σ):  2.20% ± 4.55%

       ▆██▅▃▁   ▁▂▃▃▁▁▂  ▁    ▁                               
  ▇▁▁▁▁████████████████████▇█▅█▅▇█▇▅▅▅▇▇▄▅▄▄▁▄▆▁▄▁▁▅▁▄▄▁▁▁▁▅ █
  6.38 ms      Histogram: log(frequency) by time     11.6 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

In [4]:
# complexity is n^2 + n^2 = O(n^2)
@benchmark $A * ($B * $x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  162.750 μs …   8.289 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     226.125 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   253.704 μs ± 147.541 μs  ┊ GC (mean ± σ):  0.11% ± 0.91%

     ▄▇   ▂ █     ▃█▂                                            
  ▁▁▃██▇▃▅█▂█▇▁▁▁▂███▃▃▂▂▂▂▁▂▂▂▁▂▂▁▁▁▁▁▄▄▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  163 μs           Histogram: frequency by time          504 μs <

 Memory estimate: 15.88 KiB, allocs estimate: 2.

In [5]:
# fortunately, this is parsed to the more efficient way
@code_lowered A * B * x

CodeInfo(
1 ─ %1 = B * x
│   %2 = A * %1
└──      return %2
)

## Performance of computer systems

* **FLOPS** (floating point operations per second) is a measure of computer performance. 

* For example, my laptop has the Intel i7-6920HQ (Skylake) CPU with 4 cores runing at 2.90 GHz (cycles per second).

In [6]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores
Environment:
  JULIA_EDITOR = code


Intel Skylake CPUs can do 16 DP flops per cylce and 32 SP flops per cycle. Then the **theoretical throughput** of my laptop is
$$ 4 \times 2.9 \times 10^9 \times 16 = 185.6 \text{ GFLOPS DP} $$
in double precision and
$$ 4 \times 2.9 \times 10^9 \times 32 = 371.2 \text{ GFLOPS SP} $$
in single precision. 

* In Julia, `LinearAlgebra.peakflops` computes the peak flop rate of the computer by using double precision `gemm!`

In [7]:
using LinearAlgebra

LinearAlgebra.peakflops(2^14) # matrix size 2^14

3.7697399215527997e11